In [2]:
import datasets as ds
import pandas as pd
import chess
import numpy as np
import tensorflow as tf

dataset = ds.load_dataset("laion/strategic_game_chess", split="train", streaming=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/1599 [00:00<?, ?it/s]

In [27]:
one_hot_mapping = {".": np.array([0,0,0,0,0,0]),
                   "P": np.array([1,0,0,0,0,0]),
                   "N": np.array([0,1,0,0,0,0]),
                   "B": np.array([0,0,1,0,0,0]),
                   "R": np.array([0,0,0,1,0,0]),
                   "Q": np.array([0,0,0,0,1,0]),
                   "K": np.array([0,0,0,0,0,1]),
                   "p": np.array([-1,0,0,0,0,0]),
                   "n": np.array([0,-1,0,0,0,0]),
                   "b": np.array([0,0,-1,0,0,0]),
                   "r": np.array([0,0,0,-1,0,0]),
                   "q": np.array([0,0,0,0,-1,0]),
                   "k": np.array([0,0,0,0,0,-1])}

def process_batch(games_array):
  boards = []
  moves = []
  for game in games_array:
    board = chess.Board()
    board_states = []
    board_states_labels = []
    if len(game) < 16: continue
    for move_str in game[:16]:
        move = chess.Move.from_uci(move_str)
        move_from = move.from_square
        move_to = move.to_square
        board_states_labels.append((move_from, move_to))
        board_states.append(board.copy())
        board.push(move)
    moves.append(board_states_labels[::2])
    boards.append(board_states[::2])

  boards = np.array(boards)
  moves = np.array(moves)
  boards = boards.flatten()
  moves = moves.reshape(-1, moves.shape[-1])
  return (boards, moves)

def create_tensors(boards_data, moves_labels):
  data = np.array(boards_data).astype(str)
  games_data = []
  games_labels = []
  for i in range(len(data)):
      data_split = np.array([row.split() for row in data[i].split("\n")])
      labels_split = tf.keras.utils.to_categorical(moves_labels[i], num_classes=64)
      data_encoded_state = np.zeros([8, 8, 6])
      for char, encoding in one_hot_mapping.items():
              data_encoded_state[data_split == char] = encoding
      games_data.append(np.array(data_encoded_state))
      games_labels.append(labels_split)

  train_data = tf.constant(games_data[0 : int(len(games_data) * .70)])
  train_labels = tf.constant(games_labels[0 : int(len(games_labels) * .70)])
  test_data = tf.constant(games_data[int(len(games_data) * .70) : len(games_data)])
  test_labels = tf.constant(games_labels[int(len(games_labels) * .70) : len(games_labels)])
  return train_data, train_labels, test_data, test_labels

In [29]:
input_layer = tf.keras.layers.Input(shape=(8,8,6))
x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', strides=1)(input_layer)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', strides=1)(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', strides=1)(x)
x = tf.keras.layers.Flatten()(x)
output1 = tf.keras.layers.Dense(64, activation="softmax", name="output1")(x)
output2 = tf.keras.layers.Dense(64, activation="softmax", name="output2")(x)

baseline_model = tf.keras.Model(inputs=input_layer, outputs=[output1, output2])

baseline_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'accuracy'])

In [ ]:
limited_dataset = dataset.take(100000)
batched_dataset = limited_dataset.batch(1000)
for batch in batched_dataset:
  boards, moves = process_batch(batch['Moves'])
  train_data, train_labels, test_data, test_labels = create_tensors(boards, moves)
  bsm_history = baseline_model.fit(train_data, [train_labels[:,0,:], train_labels[:,1,:]], epochs=1, validation_split=0.1)

In [35]:
def predict_from_board(board):
    board_data = np.array([board]).astype(str)
    data_split = np.array([row.split() for row in board_data[0].split("\n")])
    data_encoded_state = np.zeros([8, 8, 6])
    for char, encoding in one_hot_mapping.items():
            data_encoded_state[data_split == char] = encoding
    tensor_data = np.expand_dims(np.array(data_encoded_state), 0)
    tensor = tf.constant(tensor_data)
    test_pred = baseline_model.predict(tensor)
    move_from_pred = np.argmax(test_pred[0], axis=-1)
    move_to_pred = np.argmax(test_pred[1], axis=-1)
    return (chess.square_name(move_from_pred[0]), chess.square_name(move_to_pred[0]))

In [33]:
board = chess.Board()

In [ ]:
predict_from_board(board)

In [42]:
move = chess.Move.from_uci('e2e4')
board.push(move)